In [99]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
import pandas as pd
from numpy import genfromtxt
import itertools
import random

Read Embedding File

In [100]:
df=pd.read_csv('emb/karate.emb',skiprows=1,sep=' ',header=None)
df=df.sort_values(by=[0],ascending=True)
emb=df.as_matrix().astype(np.float32)
emb=emb[:,1:]

Read Graph

In [101]:
G=nx.read_edgelist('graph/karate.edgelist',nodetype=int)
edgeLen=len(G.edges())
edges_G=G.edges()
edges_G=np.asarray(edges_G,dtype=int)
np.random.shuffle(edges_G)

Create Egde Feature vectors for positive class

In [102]:
edge_features_1=np.empty(shape=[0,emb.shape[1]])
for edge in edges_G:
    node1=emb[edge[0]-1]
    node2=emb[edge[1]-1]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_1=np.vstack([edge_features_1,hadamard])

In [103]:
G_neg=nx.Graph()
sample_len=len(G.edges())
i=0
E=G.edges()
while(len(G_neg.edges())<sample_len):
    n1=random.choice(G.nodes())
    n2=random.choice(G.nodes())
    if(G.has_edge(n1,n2)==False):
        G_neg.add_edge(n1,n2)

In [104]:
edge_features_0=np.empty(shape=[0,emb.shape[1]])
for edge in G_neg.edges():
    node1=emb[edge[0]-1]
    node2=emb[edge[1]-1]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_0=np.vstack([edge_features_0,hadamard])

In [105]:
class_0=np.zeros([edgeLen,1])
class_1=np.ones([edgeLen,1])
edge_features_0=np.hstack([edge_features_0,class_0])
edge_features_1=np.hstack([edge_features_1,class_1])
dataset=np.vstack([edge_features_0,edge_features_1])

In [106]:
dataset.shape

(154, 129)

In [107]:
for i in range(10):
    np.random.shuffle(dataset)
X=dataset[:,:dataset.shape[1]-1]
y=dataset[:,dataset.shape[1]-1:]
y=np.reshape(y,[y.shape[0],])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [114]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [112]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [115]:
from sklearn.metrics import f1_score,precision_score,recall_score
f1=f1_score(y_test, y_pred, average='macro')
precision=precision_score(y_test, y_pred, average='macro')
recall=recall_score(y_test, y_pred, average='macro')

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [116]:
f1, precision, recall

(0.29166666666666669, 0.20588235294117646, 0.5)